지수이동평균.   

3개를 이용해서 구했고, 최소날짜는 7개가 있어야 한다고 했다.   

이해는 못했다. 일단 넣었는데 앞의 xgb보다 훨씬 좋았다.

In [155]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 진행상황
from tqdm import tqdm
from IPython.display import display

# 파생변수
import holidays as hd

In [156]:
train = pd.read_csv('/content/drive/MyDrive/colab전용/train.csv')
test = pd.read_csv('/content/drive/MyDrive/colab전용/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/colab전용/submission.csv')

In [157]:
train.head()

,store_id,date,time,card_id,amount,installments,days_of_week,holyday
0,0,2016-12-14,18:05:31,d297bba73f,5,NaN,2,0
1,0,2016-12-14,18:05:54,d297bba73f,-5,NaN,2,0
2,0,2016-12-19,12:42:31,0880849c05,144,NaN,0,0
3,0,2016-12-19,12:48:08,8b4f9e0e95,66,NaN,0,0
4,0,2016-12-19,13:31:08,7ad237eed0,24,NaN,0,0


In [158]:
train=train.groupby(['date','store_id'],as_index=False)['amount'].sum()
test=test.groupby(['date','store_id'],as_index=False)['amount'].sum()

In [159]:
train['date']=pd.to_datetime(train['date'])
test['date']=pd.to_datetime(test['date'])

In [213]:
# span=3일때 

for store in tqdm(test.store_id.unique()):
    df=test[test.store_id==store][['date','amount']]
    df=df.set_index('date')

    last=df.index[-1]
    start=last+pd.Timedelta(days=1)
    after_100days=start+pd.Timedelta(days=99)

    time_idx=pd.date_range(start=start,end=after_100days,freq='D')
    sub=pd.DataFrame(index=time_idx)
    df=pd.concat([df,sub])

    total_sales=df.amount.ewm(span=3,min_periods=7).mean()[-100:].sum()

    submission.loc[store,'store_id']=store
    submission.loc[store,'total_sales']=total_sales
display(submission)

100%|██████████| 200/200 [00:01<00:00, 158.29it/s]


,store_id,total_sales
0,0,85546.537447
1,1,34064.278671
2,2,30854.105621
3,3,45690.585541
4,4,21600.535868
...,...,...
195,195,61841.149537
196,196,155608.691855
197,197,8380.424426
198,198,22748.850332


In [215]:
# submission.to_csv('/content/drive/MyDrive/colab전용/ewm_2021-01-02.csv',index=False)

In [214]:
# dict_holidays=hd.CountryHoliday('KR',years=[2016,2017,2018])

# holidays=pd.DataFrame(columns=['date','holidays'])
# for key,value in sorted(dict_holidays.items()):
#     holidays=holidays.append({'date':key,'holidays':value},ignore_index=True)

# holidays['date']=pd.to_datetime(holidays['date'])
# display(holidays)

In [94]:
# train=pd.merge(train,holidays,on='date',how='left')
# test=pd.merge(test,holidays,on='date',how='left')

# train['holidays']=train['holidays'].fillna('no').apply(lambda date:1 if date!='no' else 0)
# test['holidays']=test['holidays'].fillna('no').apply(lambda date:1 if date!='no' else 0)